In [ ]:
from openpyxl import load_workbook
from openpyxl.cell.rich_text import CellRichText, TextBlock

workbook = load_workbook("sample_data/test_doc.xlsx", rich_text=True)

# Always working with the first sheet, index at 0.
worksheet = workbook.worksheets[0]

# (Skip the header row) iterate the row, cell by cell, left to right.
for row in worksheet.iter_rows(min_row=2): # min_row = 1 (header), 2 = first data row (skipping the header row)
    for cell in row:
        
        # If the cell is empty or cell is a string without a strike-through, enter to continue (skip this cell - do nothing with it)
        if cell.value is None or isinstance(cell.value, str) and not cell.font.strike:
            continue # continue to next iteration - doesn't proceed with next if-condition
        
        # If the entire cell is strike-through, clear the cell
        if isinstance(cell.value, str) and cell.font.strike:
            cell.value = None 
            continue # continue to next interation - doesn't proceed with next if-condition
        
        # Otherwise, if cell.value is a CellRichText, it means the content of the cell contains rich text, like coloring and strike-thrus.
        if isinstance(cell.value, CellRichText):
            filtered_runs = [] # Initiate empty list to contain runs w/o strike-thrus. 

            # Iterate through the runs in the rich text, append all runs w/o a strike-thru to filtered_runs list. 
            for run in cell.value:
                if isinstance(run, TextBlock) and run.font.strike is None and run.text != '\n':
                    run.text = run.text.strip() # Strip all '\n' 
                    run.text += '\n'            # Add '\n' to the end of the text.
                    filtered_runs.append(run)   # Add modified run to list.

                # Sometimes, run is just a str and not TextBlock.
                elif isinstance(run, str):
                    run = run.strip('\n') 
                    run += '\n'
                    filtered_runs.append(run)

            # Recreate cell.value CellRichText with the filtered_runs list that only contains runs w/o strike-thrus. 
            cell.value = CellRichText(filtered_runs)

workbook.save("sample_data/output_test.xlsx")


In [4]:
from pydantic import BaseModel
import pandas as pd

df = pd.read_excel("sample_data/SRO658Markup.xlsx", engine="openpyxl")



In [ ]:
import openpyxl, sys, pathlib, platform
print("openpyxl version  →", openpyxl.__version__)
print("module location   →", pathlib.Path(openpyxl.__file__).as_posix())
print("python executable →", sys.executable)
print("python version    →", platform.python_version())


In [ ]:
from pydantic import BaseModel
import pandas as pd

df = pd.read_excel("sample_data/SRO658_DOORS.xlsx", engine="openpyxl")

df.columns
# df['Object Text']

In [ ]:
from pydantic import BaseModel
import pandas as pd

# Read and load the markup into dataframe.
df_markup = pd.read_excel("sample_data/SRO658Markup.xlsx", engine="openpyxl")
# Read and load the export from DOORS into dataframe.
df_doors = pd.read_excel("sample_data/SRO658_DOORS.xlsx", engine="openpyxl")

df_compare = df_markup['Object Text'].compare(df_doors['Object Text'], align_axis="index")
df_compare.infer_objects
# print(df_compare['self'])
# print(df_compare['other'])
